클러스터 재분배 테스트

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans

from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings("ignore")

In [38]:
PATH = r'C:\Users\Wyatt\wyatt37\data\energy'

In [39]:
train_col = ['num', 'date_time', 'target', 'temp', 'wind', 'humid', 'rain', 'sunny', 'cooler', 'solar']
test_col = ['num', 'date_time', 'temp', 'wind', 'humid', 'rain', 'sunny', 'cooler', 'solar']

In [40]:
train = pd.read_csv(PATH + '\\train.csv', encoding='cp949', names=train_col, header=0)
test = pd.read_csv(PATH + '\\test.csv', encoding='cp949', names=test_col, header=0)
sub = pd.read_csv(PATH + '\\sample_submission.csv', encoding='cp949')

## Preprocessing

### datetime

datetime 과 관련된 컬럼을 만들어 줍니다.

In [41]:
def make_time(train):
    train['date_time'] = pd.to_datetime(train.date_time)
    
    # dt.
    train['month'] = train.date_time.dt.month                    # 월(숫자)
    train['day'] = train.date_time.dt.day                        # 일(숫자)
    train['hour'] = train.date_time.dt.hour                      # 시(숫자)
    train['weekday'] = train.date_time.dt.weekday                # 시(숫자)
    train['dayofyear'] = train.date_time.dt.dayofyear            # 연 기준 몇일째(숫자)
    train['weekofyear'] = train.date_time.dt.isocalendar().week  # 연 기준 몇주째(숫자) (=week)
    train['month_start'] = train.date_time.dt.is_month_start     # 월 시작 일
    train['month_end'] = train.date_time.dt.is_month_end         # 월 마감 일
    
    # astype
    train.weekofyear = train.weekofyear.astype(int)
    
    # drop datetime
    train.drop(['date_time'], axis=1, inplace=True)
    
    return train

In [42]:
train = make_time(train)
test = make_time(test)

### holiday

2020년 6~8월에는 공휴일이 많지는 않았습니다. 주말과 함께 구분해줍니다.

In [43]:
# 6월 6일은 현충일
# 8월 15일은 광복절
# 8월 17일은 임시공휴일

# 휴일이 많지 않으니, 주말에 포함시키자.

train.loc[train.weekday >= 5, 'holiday'] = True
train.loc[train.weekday < 5, 'holiday'] = False
train.loc[(train.month == 8) & (train.day == 17), 'holiday'] = True

test.loc[test.weekday >= 5, 'holiday'] = True
test.loc[test.weekday < 5, 'holiday'] = False

### Cooler & Solar

테스트 셋에는 Cooler와 Solar가 비어 있습니다. 다른 데이터와 함께 채워줍니다.

In [44]:
# test에 cooler와 solar를 넣어주겠습니다.
for i in range(1, 61):
    test.loc[test.num == i, 'cooler'] = train.loc[train.num == i, 'cooler'].mean()
    test.loc[test.num == i, 'solar'] = train.loc[train.num == i, 'solar'].mean()

In [45]:
# test set의 빈 값을 채워줍니다.
test[['temp', 'wind', 'humid', 'rain', 'sunny']] = test[['temp', 'wind', 'humid', 'rain', 'sunny']].interpolate(method='values').round(1)

### Clustering

In [46]:
from sklearn.cluster import KMeans

In [47]:
weekday_mean = (
    train.groupby(['num', 'weekday'])['target'].mean()
    .reset_index()
    .pivot('num', 'weekday', 'target')
    .reset_index()
)

In [48]:
weekday_std = (
    train.groupby(['num', 'weekday'])['target'].std()
    .reset_index()
    .pivot('num', 'weekday', 'target')
    .reset_index()
    .drop('num', axis=1)
)

In [49]:
hour_mean = (
    train.groupby(['num', 'hour'])['target'].mean()
    .reset_index()
    .pivot('num', 'hour', 'target')
    .reset_index()
    .drop('num', axis=1)
)

In [50]:
hour_std = (
    train.groupby(['num', 'hour'])['target'].std()
    .reset_index()
    .pivot('num', 'hour', 'target')
    .reset_index()
    .drop('num', axis=1)
)

In [51]:
cl_df = pd.concat([weekday_mean, hour_mean, weekday_std, hour_std], axis=1)

In [52]:
columns = (
    ['num']
    + ['day_mean_' + str(i) for i in range(7)]
    + ['hour_mean_' + str(i) for i in range(24)]
    + ['day_std_' + str(i) for i in range(7)]
    + ['hour_std_' + str(i) for i in range(24)]
)

In [53]:
cl_df.columns = columns

In [54]:
for i in range(len(cl_df)):
    cl_df.iloc[i, 1:8] = (cl_df.iloc[i, 1:8] - cl_df.iloc[i, 1:8].mean()) / cl_df.iloc[i, 1:8].std()
    cl_df.iloc[i, 8:32] = (cl_df.iloc[i, 8:32] - cl_df.iloc[i, 8:32].mean()) / cl_df.iloc[i, 8:32].std()
    cl_df.iloc[i, 32:39] = (cl_df.iloc[i, 32:39] - cl_df.iloc[i, 32:39].mean()) / cl_df.iloc[i, 32:39].std()
    cl_df.iloc[i, 39:] = (cl_df.iloc[i, 39:] - cl_df.iloc[i, 39:].mean()) / cl_df.iloc[i, 39:].std()

In [55]:
model = KMeans(n_clusters = 4, random_state = 42)
pred = model.fit_predict(cl_df.iloc[:, 1:])

In [56]:
cl_df['km_cluster'] = pred

In [57]:
train_cl = pd.merge(train, cl_df[['num', 'km_cluster']], how='left', on='num')

In [58]:
test_cl = test.copy()

In [59]:
# cooler와 solar를 넣어주겠습니다.
for i in range(1, 61):
    test_cl.loc[test_cl.num == i, 'km_cluster'] = (
        train_cl.loc[train_cl.num == i, 'km_cluster'].max()
    )

육안으로 확인한 조금 다른 건물들을 따로 빼주겠습니다.

- 4:7, 15, 18, 57
- 5:38, 58
- 6:34

In [60]:
train_cl.loc[train_cl.num == 7, 'km_cluster'] = 4
train_cl.loc[train_cl.num == 15, 'km_cluster'] = 4
train_cl.loc[train_cl.num == 18, 'km_cluster'] = 4
train_cl.loc[train_cl.num == 57, 'km_cluster'] = 4

test_cl.loc[test_cl.num == 7, 'km_cluster'] = 4
test_cl.loc[test_cl.num == 15, 'km_cluster'] = 4
test_cl.loc[test_cl.num == 18, 'km_cluster'] = 4
test_cl.loc[test_cl.num == 57, 'km_cluster'] = 4

In [61]:
train_cl.loc[train_cl.num == 38, 'km_cluster'] = 5
train_cl.loc[train_cl.num == 58, 'km_cluster'] = 5

test_cl.loc[test_cl.num == 38, 'km_cluster'] = 5
test_cl.loc[test_cl.num == 58, 'km_cluster'] = 5

In [62]:
train_cl.loc[train_cl.num == 34, 'km_cluster'] = 6

test_cl.loc[test_cl.num == 34, 'km_cluster'] = 6

#### train, test 찢기

In [63]:
train_cl_0 = train_cl.loc[train_cl.km_cluster == 0,
                          ~train_cl.columns.isin(['target', 'km_cluster'])].reset_index(drop=True)
train_cl_1 = train_cl.loc[train_cl.km_cluster == 1,
                          ~train_cl.columns.isin(['target', 'km_cluster'])].reset_index(drop=True)
train_cl_2 = train_cl.loc[train_cl.km_cluster == 2,
                          ~train_cl.columns.isin(['target', 'km_cluster'])].reset_index(drop=True)
train_cl_3 = train_cl.loc[train_cl.km_cluster == 3,
                          ~train_cl.columns.isin(['target', 'km_cluster'])].reset_index(drop=True)
train_cl_4 = train_cl.loc[train_cl.km_cluster == 4,
                          ~train_cl.columns.isin(['target', 'km_cluster'])].reset_index(drop=True)
train_cl_5 = train_cl.loc[train_cl.km_cluster == 5,
                          ~train_cl.columns.isin(['target', 'km_cluster'])].reset_index(drop=True)
train_cl_6 = train_cl.loc[train_cl.km_cluster == 6,
                          ~train_cl.columns.isin(['target', 'km_cluster'])].reset_index(drop=True)

In [64]:
train_cl_0.shape, train_cl_1.shape, train_cl_2.shape, train_cl_3.shape, train_cl_4.shape, train_cl_5.shape, train_cl_6.shape

((61200, 17),
 (20400, 17),
 (8160, 17),
 (18360, 17),
 (8160, 17),
 (4080, 17),
 (2040, 17))

In [65]:
train_cl_0_y = train_cl.loc[train_cl.km_cluster == 0, 'target'].reset_index(drop=True)
train_cl_1_y = train_cl.loc[train_cl.km_cluster == 1, 'target'].reset_index(drop=True)
train_cl_2_y = train_cl.loc[train_cl.km_cluster == 2, 'target'].reset_index(drop=True)
train_cl_3_y = train_cl.loc[train_cl.km_cluster == 3, 'target'].reset_index(drop=True)
train_cl_4_y = train_cl.loc[train_cl.km_cluster == 4, 'target'].reset_index(drop=True)
train_cl_5_y = train_cl.loc[train_cl.km_cluster == 5, 'target'].reset_index(drop=True)
train_cl_6_y = train_cl.loc[train_cl.km_cluster == 6, 'target'].reset_index(drop=True)

In [66]:
train_cl_0_y.shape, train_cl_1_y.shape, train_cl_2_y.shape, train_cl_3_y.shape, train_cl_4_y.shape, train_cl_5_y.shape, train_cl_6_y.shape

((61200,), (20400,), (8160,), (18360,), (8160,), (4080,), (2040,))

In [67]:
test_cl_0 = test_cl.loc[test_cl.km_cluster == 0, ~test_cl.columns.isin(['km_cluster'])]
test_cl_1 = test_cl.loc[test_cl.km_cluster == 1, ~test_cl.columns.isin(['km_cluster'])]
test_cl_2 = test_cl.loc[test_cl.km_cluster == 2, ~test_cl.columns.isin(['km_cluster'])]
test_cl_3 = test_cl.loc[test_cl.km_cluster == 3, ~test_cl.columns.isin(['km_cluster'])]
test_cl_4 = test_cl.loc[test_cl.km_cluster == 4, ~test_cl.columns.isin(['km_cluster'])]
test_cl_5 = test_cl.loc[test_cl.km_cluster == 5, ~test_cl.columns.isin(['km_cluster'])]
test_cl_6 = test_cl.loc[test_cl.km_cluster == 6, ~test_cl.columns.isin(['km_cluster'])]

In [68]:
test_cl_0.shape, test_cl_1.shape, test_cl_2.shape, test_cl_3.shape, test_cl_4.shape, test_cl_5.shape, test_cl_6.shape

((5040, 17),
 (1680, 17),
 (672, 17),
 (1512, 17),
 (672, 17),
 (336, 17),
 (168, 17))

In [69]:
train_x_sets = [train_cl_0, train_cl_1, train_cl_2, train_cl_3, train_cl_4, train_cl_5, train_cl_6]
train_y_sets = [train_cl_0_y, train_cl_1_y, train_cl_2_y, train_cl_3_y, train_cl_4_y, train_cl_5_y, train_cl_6_y]
test_x_sets = [test_cl_0, test_cl_1, test_cl_2, test_cl_3, test_cl_4, test_cl_5, test_cl_6]

In [70]:
preds = []
fe = []
for x, y, t in tqdm(zip(train_x_sets, train_y_sets, test_x_sets)):
    rf = RandomForestRegressor(random_state=42)
    rf.fit(x, y)
    pred = rf.predict(t)
    preds.append(pred)
    fe.append(rf.feature_importances_)

7it [00:43,  6.25s/it]


In [71]:
pred_set = []
for t, p in zip(test_x_sets, preds):
    t['pred'] = p
    pred_set.append(t)

In [72]:
pred_set = pd.concat(pred_set).sort_index()

In [73]:
sub.answer = pred_set.pred

In [922]:
sub.to_csv('submission/210608_rf-vanila_clusters-and-buildings.csv', index=False)
sub

,num_date_time,answer
0,1 2020-08-25 00,8671.87296
1,1 2020-08-25 01,8669.97432
2,1 2020-08-25 02,8669.09304
3,1 2020-08-25 03,8666.65656
4,1 2020-08-25 04,8656.40520
...,...,...
10075,60 2020-08-31 19,4089.12192
10076,60 2020-08-31 20,3981.68352
10077,60 2020-08-31 21,3786.92928
10078,60 2020-08-31 22,3510.26352


In [74]:
sub.to_csv('submission/210609_rf-vanila_clusters-and-buildings+holiday.csv', index=False)
sub

,num_date_time,answer
0,1 2020-08-25 00,8675.19720
1,1 2020-08-25 01,8671.44528
2,1 2020-08-25 02,8670.75192
3,1 2020-08-25 03,8666.96760
4,1 2020-08-25 04,8657.15040
...,...,...
10075,60 2020-08-31 19,4091.33808
10076,60 2020-08-31 20,3979.33344
10077,60 2020-08-31 21,3789.91440
10078,60 2020-08-31 22,3534.15744


In [76]:
warnings.simplefilter(action='ignore', category=FutureWarning)
plt.rcParams["font.size"] = "14"

fe_avg = pd.DataFrame(np.mean(fe, axis=0),
                      index=x.columns, columns=['fe']).sort_values(by=['fe'], ascending=False)

plt.figure(figsize=(16, 8))
plots = sns.barplot(fe_avg.fe, y=fe_avg.index)

# Annotate every single Bar with its value, based on it's width           
for bar in plots.patches:
    width = bar.get_width()
    plt.text(0.015+ bar.get_width(),
             bar.get_y() + 0.5 * bar.get_height(),
             '{:.3f}'.format(width),
             ha='center', va='center')

plt.show()